<a href="https://colab.research.google.com/github/shacharmirkin/misc/blob/main/country_city.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Country-city (Eretz-Ir)

(Just playing a bit with mapping)

This notebook shows a map of the world countries, and when you hover over a country, 
you see a city, an animal, a plant and an object, all starting with the same letter.

<img src="https://github.com/shacharmirkin/misc/blob/main/images/country-city-ex.png?raw=1" width=500>

The main libraries I used:
- http://python-visualization.github.io/folium
- https://geopandas.org
- https://nltk.org/howto/wordnet


### Setup

In [ ]:
! pip install --quiet geopandas nltk
import geopandas
import matplotlib.pyplot as plt

from collections import defaultdict
import random
import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet');

### Geopanda's datasets

In [ ]:
# List built-in Geopanda datasets
geopandas.datasets.available

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
cities = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))
cities.head()

## Country-City (Animal-Plant-Object)
For each country, we show a city, an animal, a plant and an object that start with the same letter.
Originally I wanted to show a different icons for each animal, but supported free icons were not available

#### Adding city info from the internal dataset

In [ ]:
type_to_inst = defaultdict(dict)
# Add the cities from the dataset
# No cities in this dataset for some letters so I'm adding them manually
city_list = list(cities.name.values)
city_list.extend(['Zurich', 'Qionghai', 'Ubatuba', 'El Paso'])

type_to_inst['city'] = defaultdict(list)
for city in city_list:
  type_to_inst['city'][city[0].upper()].append(city)

### Retrieving info from WordNet
Collecting isntances for each of animal, plant, object

In [ ]:
# The synonym number can be found in http://wordnetweb.princeton.edu/perl/webwn

types = ['animal','plant', 'object']
animal = wn.synset('animal.n.01')
plant = wn.synset('plant.n.02')
object_ = wn.synset('object.n.01')

hypo = lambda s: s.hyponyms()

# Structure example: 'Blue Orchid' is in dict_['plant']['B']

for type_, synset in zip(types, [animal, plant, object_]):
  curr_dict = defaultdict(list)
  clousure = list(synset.closure(hypo, depth=-1))
  for c in clousure:
    if c.instance_hyponyms() == []:
      name = c.lemmas()[0].name().replace('_', ' ').title()
      curr_dict[name[0].upper()].append(name)

  type_to_inst[type_] = curr_dict

### Showing the data on the map

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

# make sure dataframe coordinates are in sync with geographic coordinates (lat, long)
# 4326 is just the EPSG identifier of WGS84. ESPG https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset
cities.to_crs(epsg=4326)
world.to_crs(epsg=4326)

berlin_loc = cities.query('name=="Berlin"').geometry.values[0]  # from the cities dataset

# Create a map with centered around Berlin
# Coordinates are represented as latitude, longtitude => y,x (and not x,y)
m = folium.Map(location=[berlin_loc.y, berlin_loc.x], zoom_start=4, tiles="Stamen Terrain")

# Overlay the boundaries of boroughs
for _, row in world.iterrows():
    # without simplifying the representation of each borough, the map might not be displayed
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)  # the higher the tolerance, the less accurate
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

# Mark Grenoble on the map
grenoble_loc = [45.188614872721125, 5.724174129578328]  # from Google maps
folium.Marker(location=grenoble_loc).add_to(m)

# Calculate centroids of each country (centroid of the points in the polygons)
# Note: centroid may be outside the boundaries when there are multiple polygons representing the country, e.g. France

world['lat'] = world.centroid.y
world['long'] = world.centroid.x

# style prefix and suffix
pre = '<span style="color:#4d94ff;">'
suf = '</span>'

for _, row in world.iterrows():
  tooltip_text = pre + 'COUNTRY: ' + suf + row['name'] + '<br>'
  f_letter = row['name'][0].upper()
  for k in type_to_inst.keys():
    instances = type_to_inst[k][f_letter]
    if len(instances) > 0:
      tooltip_text += pre + k.upper() + ': ' +suf + random.choice(instances) + '<br>'

  folium.Marker(location=[row['lat'], row['long']],
                icon=folium.Icon(prefix="fa", icon="paw"),
                tooltip=folium.Tooltip(tooltip_text, style="font-weight: bold;font-size:130%;")
                ).add_to(m)

m